In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as skl
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,precision_score,recall_score
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import time
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import metrics

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, BatchNormalization
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor,KerasClassifier
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from livelossplot import PlotLossesKerasTF

RANDOM_SEED = 13122001

print("TensorFlow version:",tf.__version__)

In [ ]:
X_train= pd.read_csv('X_train.csv')
X_test=pd.read_csv('X_test.csv')
y_test=pd.read_csv('y_test.csv')
y_train=pd.read_csv('y_train.csv')

In [ ]:
def build_model(activation='relu',learning_rate=0.01,nodes=[8,16,8]):
    model=Sequential()
    model.add(Dense(nodes[0],input_dim=len(X_test.columns),activation=activation))
    for i in range(1,len(nodes)):
      model.add(Dense(nodes[i],activation=activation))
    model.add(Dense(1,activation='sigmoid'))
    
    model.compile(
        loss='binary_crossentropy',
        optimizer = tf.optimizers.Adam(learning_rate),
        metrics = ['accuracy'])
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
#,'sigmoid','tanh'

TUNING_DICT = {
    'activation':['relu'],
    'learning_rate':[0.001],
    'nodes':[[256,512,512,256]]
}

In [ ]:
kf = KFold(n_splits=2,shuffle=True,random_state=RANDOM_SEED)

model = KerasClassifier(build_fn=build_model,epochs=10,batch_size=32)
grid_search = GridSearchCV(estimator = model,
                           param_grid = TUNING_DICT,
                           cv = kf,
                           scoring = 'accuracy',
                           refit='True',
                           verbose=1
                          )
grid_search.fit(X_train,y_train,validation_split=0.3)

In [ ]:
print("Best: %f using %s"% (grid_search.best_score_,grid_search.best_params_))
means= grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
params = grid_search.cv_results_['params']
for mean,stdev,param in zip(means,stds,params):
    print("%f (%f) with: %r"%(mean,stdev,param))

In [ ]:
best_mlp_model = grid_search.best_estimator_

In [ ]:
best_mlp_model.fit(X_train,y_train
                   , epochs=20
                   ,validation_data=(X_test,y_test)
                   ,callbacks=[PlotLossesKerasTF()],verbose=1)

In [ ]:
print(confusion_matrix(y_test,best_mlp_model.predict(X_test)))

In [ ]:
predictions= best_mlp_model.predict(X_test)
print('Accuracy',metrics.accuracy_score(y_test,predictions))